# CS375 - Tutorial 2

### Welcome to tutorial 2! This tutorial will introduce you to unsupervised learning methods, specifically how to train a sparse autoencoder and variational autoencoder on MNIST, and how to evaluate the trained model on neural data. As before everything will be implemented using TFUtils. We will start with a sparse autoencoder and then move on to variational autoencoders.

## 1.) Training and evaluating a sparse autoencoder on MNIST

### 1.1.) Define a simple sparse autoencoder consisting of one fully connected layer in the encoder and one fully connected layer in the decoder. The input dimension is 784. Use xavier initialization and l1-regularization on the weights, initialize all biases to 0 and use a tanh activation function:

In [ ]:
%matplotlib inline
from __future__ import division
from tfutils import base, data, optimizer, utils

import numpy as np
import tensorflow as tf
import os
import pymongo as pm
import matplotlib.pyplot as plt
import scipy.signal as signal

# connect to database
dbname = 'mnist'
collname = 'autoencoder'
port = 24444
conn = pm.MongoClient(port = port)
coll = conn[dbname][collname + '.files']

def sparse_autoencoder(inputs, train=True, beta = 5e-4, n_hidden = 100, **kwargs): 
    '''
    Implements a simple autoencoder consisting of two fully connected layers
    '''
    # flatten the input images
    inp = tf.reshape(inputs['images'], [inputs['images'].get_shape().as_list()[0], -1])
    
    ### YOUR CODE HERE

    return output, {}

### 1.2.) Define the l2 loss function for the sparse autoencoder:

In [ ]:
def sparse_autoencoder_loss(inputs, outputs, **kwargs):
    '''
    Defines the loss = l2(inputs - outputs) + l1(weights)
    '''
    # flatten the input images
    inputs = tf.reshape(inputs, [inputs.get_shape().as_list()[0], -1])
    
    ### YOUR CODE HERE
    
    return loss

def sparse_autoencoder_validation(inputs, outputs, **kwargs):
    '''
    Wrapper for using the loss function as a validation target
    '''
    return {'l2_loss': sparse_autoencoder_loss(inputs['images'], outputs),
            'pred': outputs,
            'gt': inputs['images']}

### Now let's define and run our sparse autoencoder experiment on MNIST in TFUtils. We will use the Adam optimizer and an exponentially decaying learning rate:

In [ ]:
def online_agg_mean(agg_res, res, step):
    """
    Appends the mean value for each key
    """
    if agg_res is None:
        agg_res = {k: [] for k in res}
    for k, v in res.items():
        if k in ['pred', 'gt']:
            value = v
        else:
            value = np.mean(v)
        agg_res[k].append(value)
    return agg_res

def agg_mean(results):
    for k in results:
        if k in ['pred', 'gt']:
            results[k] = results[k][0]
        elif k is 'l2_loss':
            results[k] = np.mean(results[k])
        else:
            raise KeyError('Unknown target')
    return results

# number of hidden neurons
n_hidden = 100
# scaling of l1 regularization
beta = 1e-2 # 1e-4 = no regularization

params = {}

params['load_params'] = {
    'do_restore': False,
}

params['save_params'] = {
    'host': 'localhost',
    'port': 24444,
    'dbname': 'mnist',
    'collname': 'autoencoder',
    'exp_id': 'exp1',
    'save_metrics_freq': 200,
    'save_valid_freq': 200,
    'save_filters_freq': 1000,
    'cache_filters_freq': 1000,
}

params['train_params'] = {
    'validate_first': False,
    'data_params': {'func': data.MNIST,
                    'batch_size': 256,
                    'group': 'train',
                    'n_threads': 1},
    'queue_params': {'queue_type': 'random',
                     'batch_size': 256},
    'num_steps': 4000,
    'thres_loss': float("inf"),
}

params['validation_params'] = {'valid0': {
    'data_params': {'func': data.MNIST,
                    'batch_size': 100,
                    'group': 'test',
                    'n_threads': 1},
    'queue_params': {'queue_type': 'fifo',
                     'batch_size': 100},
    'num_steps': 100,
    'targets': {'func': sparse_autoencoder_validation},
    'online_agg_func': online_agg_mean,
    'agg_func': agg_mean,
}}

params['model_params'] = {
    'func': sparse_autoencoder,
    'beta': beta,
    'n_hidden': n_hidden,
} 

params['learning_rate_params'] = {
    'learning_rate': 5e-3,
    'decay_steps': 2000,
    'decay_rate': 0.95,
    'staircase': True,
}

params['optimizer_params'] = {
    'func': optimizer.ClipOptimizer,
    'optimizer_class': tf.train.AdamOptimizer,
    'clip': False,
}

params['loss_params'] = {
    'targets': ['images'],
    'loss_per_case_func': sparse_autoencoder_loss,
    'loss_per_case_func_params' : {'_outputs': 'outputs', '_targets_$all': 'inputs'},
    'agg_func': tf.reduce_mean,
}

params['skip_check'] = True

coll.remove({'exp_id' : 'exp1'}, {'justOne': True})
base.train_from_params(**params)

### Now let's have a look at our training and validation curves that were stored in our database:

In [ ]:
def get_losses(exp_id):
    """
    Gets all loss entries from the database and concatenates them into a vector
    """
    q_train = {'exp_id' : exp_id, 'train_results' : {'$exists' : True}}
    return np.array([_r['loss'] 
                     for r in coll.find(q_train, projection = ['train_results']) 
                     for _r in r['train_results']])

def plot_train_loss(exp_id, start_step=None, end_step=None, N_smooth = 100, plt_title = None):
    """
    Plots the training loss
    
    You will need to EDIT this part.
    """
    # get the losses from the database
    loss = get_losses(exp_id)

    if start_step is None:
        start_step = 0
    if end_step is None:
        end_step = len(loss)
    if plt_title is None:
        plt_title = exp_id
    
    # Only plot selected loss window
    loss = loss[start_step:end_step]
    
    # plot loss
    fig = plt.figure(figsize = (15, 6))
    plt.plot(loss)
    plt.title(plt_title + ' training: loss')
    plt.grid()
    axes = plt.gca()

    # plot smoothed loss
    smoothed_loss = signal.convolve(loss, np.ones((N_smooth,)))[N_smooth : -N_smooth] / float(N_smooth)
    plt.figure(figsize = (15, 6))
    plt.plot(smoothed_loss)
    plt.title(plt_title + ' training: loss smoothed')
    plt.grid()
    axes = plt.gca()

plot_train_loss('exp1')

In [ ]:
def get_validation_data(exp_id):
    """
    Gets the validation data from the database (except for gridfs data)
    """
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates' : {'$exists' : False}}
    val_steps = coll.find(q_val, projection = ['validation_results'])
    return [val_steps[i]['validation_results']['valid0']['l2_loss'] 
            for i in range(val_steps.count())]

def plot_validation_results(exp_id, plt_title = None):
    """
    Plots the validation results i.e. the top1 and top5 accuracy
    
    You will need to EDIT this part.
    """
    # get the data from the database
    l2_loss = get_validation_data(exp_id)
    
    if plt_title is None:
        plt_title = exp_id
    
    # plot top1 accuracy
    fig = plt.figure(figsize = (15, 6))
    plt.plot(l2_loss)
    plt.title(plt_title + ' validation: l2 loss')
    plt.grid()
    axes = plt.gca()

plot_validation_results('exp1')

### Finally let's plot some example outputs of our autoencoder. The images to the left are the outputs. The images to the right are the inputs.

In [ ]:
def get_validation_images(exp_id):
    """
    Gets the validation images from the database
    """
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates' : {'$exists' : False}}
    val_steps = coll.find(q_val, projection = ['validation_results'])
    pred = np.array([val_steps[i]['validation_results']['valid0']['pred'] 
            for i in range(val_steps.count())])
    gt = np.array([val_steps[i]['validation_results']['valid0']['gt'] 
            for i in range(val_steps.count())])
    return {'gt': gt, 'pred': pred}

def plot_validation_images(exp_id, n_images = 24):
    '''
    Plots n_images images in a grid. The ground truth image is on the left 
    and the prediction is on the right.
    '''
    imgs = get_validation_images(exp_id)
    fig = plt.figure(figsize=(16, 16))
    for i in range(n_images):
        pred = np.reshape(imgs['pred'][0,i], [28, 28])
        plt.subplot(n_images/4,n_images/3,1 + i*2)
        plt.imshow(pred, cmap='gray')
        ax = plt.gca()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
        gt = np.reshape(imgs['gt'][0,i], [28, 28])
        plt.subplot(n_images/4,n_images/3,2 + i*2)
        plt.imshow(gt, cmap='gray')
        ax = plt.gca()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
plot_validation_images('exp1')

## 2.) Training and evaluating a variational autoencoder on MNIST
### 2.1.) Define a simple variational autoencoder as discussed in class consisting of  
### - one fully connected layer (dimension n_hidden) in the encoder that results in the latent variable, 
### - two fully connected layers  (dimension n_latent) that take the latent variable and reparametrize the latent distribution with it's mean mu and log of the standard deviation, and
### - two fully connected layers  (dimension n_hidden & output_dim) in the decoder that take the reparametrized latent variable and decode it into the output prediction. 
### The input dimension is 784. Use tanh activation functions in the intermediate layers and a sigmoid at the top layer. Use xavier initialization for the weights and constant initialization to 0 for the biases:

In [ ]:
def variational_autoencoder(inputs, train=True, n_hidden = 100, n_latent = 20, **kwargs): 
    '''
    Implements a simple autoencoder consisting of two fully connected layers
    '''
    outputs = {}
    # flatten the input images
    inp = tf.reshape(inputs['images'], [inputs['images'].get_shape().as_list()[0], -1])
    
    ### YOUR CODE HERE
        
    return outputs, {}

### 2.2.) Define the loss for our variational autoencoder as discussed in class:

In [ ]:
def variational_autoencoder_loss(inputs, outputs, **kwargs):
    '''
    Defines the loss = l2(inputs - outputs) + l2(weights)
    '''
    # flatten the input images
    inputs = tf.reshape(inputs, [inputs.get_shape().as_list()[0], -1])
    
    ### YOUR CODE HERE
    
    return loss

def variational_autoencoder_validation(inputs, outputs, **kwargs):
    '''
    Wrapper for using the loss function as a validation target
    '''
    return {'l2_loss': variational_autoencoder_loss(inputs['images'], outputs),
            'pred': outputs['pred'],
            'gt': inputs['images']}

In [ ]:
def online_agg_mean(agg_res, res, step):
    """
    Appends the mean value for each key
    """
    if agg_res is None:
        agg_res = {k: [] for k in res}
    for k, v in res.items():
        if k in ['pred', 'gt']:
            value = v
        else:
            value = np.mean(v)
        agg_res[k].append(value)
    return agg_res

def agg_mean(results):
    for k in results:
        if k in ['pred', 'gt']:
            results[k] = results[k][0]
        elif k is 'l2_loss':
            results[k] = np.mean(results[k])
        else:
            raise KeyError('Unknown target')
    return results

# number of hidden neurons
n_hidden = 100
# dimension of latent space
n_latent = 20

params = {}

params['load_params'] = {
    'do_restore': False,
}

params['save_params'] = {
    'host': 'localhost',
    'port': 24444,
    'dbname': 'mnist',
    'collname': 'autoencoder',
    'exp_id': 'exp2',
    'save_metrics_freq': 200,
    'save_valid_freq': 200,
    'save_filters_freq': 1000,
    'cache_filters_freq': 1000,
}

params['train_params'] = {
    'validate_first': False,
    'data_params': {'func': data.MNIST,
                    'batch_size': 256,
                    'group': 'train',
                    'n_threads': 1},
    'queue_params': {'queue_type': 'random',
                     'batch_size': 256},
    'num_steps': 10000,
    'thres_loss': float("inf"),
}

params['validation_params'] = {'valid0': {
    'data_params': {'func': data.MNIST,
                    'batch_size': 100,
                    'group': 'test',
                    'n_threads': 1},
    'queue_params': {'queue_type': 'fifo',
                     'batch_size': 100},
    'num_steps': 100,
    'targets': {'func': variational_autoencoder_validation},
    'online_agg_func': online_agg_mean,
    'agg_func': agg_mean,
}}

params['model_params'] = {
    'func': variational_autoencoder,
    'n_latent': n_latent,
    'n_hidden': n_hidden,
} 

params['learning_rate_params'] = {
    'learning_rate': 5e-3,
    'decay_steps': 10000,
    'decay_rate': 0.95,
    'staircase': True,
}

params['optimizer_params'] = {
    'func': optimizer.ClipOptimizer,
    'optimizer_class': tf.train.AdamOptimizer,
    'clip': True,
}

params['loss_params'] = {
    'targets': ['images'],
    'loss_per_case_func': variational_autoencoder_loss,
    'loss_per_case_func_params' : {'_outputs': 'outputs', '_targets_$all': 'inputs'},
    'agg_func': tf.reduce_mean,
}

params['skip_check'] = True

coll.remove({'exp_id' : 'exp2'}, {'justOne': True})
base.train_from_params(**params)

### Now let's have a look at our training and validation curves that were stored in our database:

In [ ]:
def get_losses(exp_id):
    """
    Gets all loss entries from the database and concatenates them into a vector
    """
    q_train = {'exp_id' : exp_id, 'train_results' : {'$exists' : True}}
    return np.array([_r['loss'] 
                     for r in coll.find(q_train, projection = ['train_results']) 
                     for _r in r['train_results']])

def plot_train_loss(exp_id, start_step=None, end_step=None, N_smooth = 100, plt_title = None):
    """
    Plots the training loss
    
    You will need to EDIT this part.
    """
    # get the losses from the database
    loss = get_losses(exp_id)

    if start_step is None:
        start_step = 0
    if end_step is None:
        end_step = len(loss)
    if plt_title is None:
        plt_title = exp_id
    
    # Only plot selected loss window
    loss = loss[start_step:end_step]
    
    # plot loss
    fig = plt.figure(figsize = (15, 6))
    plt.plot(loss)
    plt.title(plt_title + ' training: loss')
    plt.grid()
    axes = plt.gca()

    # plot smoothed loss
    smoothed_loss = signal.convolve(loss, np.ones((N_smooth,)))[N_smooth : -N_smooth] / float(N_smooth)
    plt.figure(figsize = (15, 6))
    plt.plot(smoothed_loss)
    plt.title(plt_title + ' training: loss smoothed')
    plt.grid()
    axes = plt.gca()

plot_train_loss('exp2')

In [ ]:
def get_validation_data(exp_id):
    """
    Gets the validation data from the database (except for gridfs data)
    """
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates' : {'$exists' : False}}
    val_steps = coll.find(q_val, projection = ['validation_results'])
    return [val_steps[i]['validation_results']['valid0']['l2_loss'] 
            for i in range(val_steps.count())]

def plot_validation_results(exp_id, plt_title = None):
    """
    Plots the validation results i.e. the top1 and top5 accuracy
    
    You will need to EDIT this part.
    """
    # get the data from the database
    l2_loss = get_validation_data(exp_id)
    
    if plt_title is None:
        plt_title = exp_id
    
    # plot top1 accuracy
    fig = plt.figure(figsize = (15, 6))
    plt.plot(l2_loss)
    plt.title(plt_title + ' validation: l2 loss')
    plt.grid()
    axes = plt.gca()

plot_validation_results('exp2')

### Finally let's plot some example outputs of our autoencoder. The images to the left are the outputs. The images to the right are the inputs.

In [ ]:
def get_validation_images(exp_id):
    """
    Gets the validation images from the database
    """
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates' : {'$exists' : False}}
    val_steps = coll.find(q_val, projection = ['validation_results'])
    pred = np.array([val_steps[i]['validation_results']['valid0']['pred'] 
            for i in range(val_steps.count())])
    gt = np.array([val_steps[i]['validation_results']['valid0']['gt'] 
            for i in range(val_steps.count())])
    return {'gt': gt, 'pred': pred}

def plot_validation_images(exp_id, n_images = 24):
    '''
    Plots n_images images in a grid. The ground truth image is on the left 
    and the prediction is on the right.
    '''
    imgs = get_validation_images(exp_id)
    fig = plt.figure(figsize=(16, 16))
    for i in range(n_images):
        pred = np.reshape(imgs['pred'][0,i], [28, 28])
        plt.subplot(n_images/4,n_images/3,1 + i*2)
        plt.imshow(pred, cmap='gray')
        ax = plt.gca()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
        gt = np.reshape(imgs['gt'][0,i], [28, 28])
        plt.subplot(n_images/4,n_images/3,2 + i*2)
        plt.imshow(gt, cmap='gray')
        ax = plt.gca()
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
plot_validation_images('exp2')